<a href="https://colab.research.google.com/github/tarangga/Machine-Learning/blob/main/Experiment_Covid_Classification_pretrained_indobert_base_uncased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [ ]:
def sheet2csv(url, save_path):
  import requests
  resp = requests.get(url)

  output = open(save_path, 'wb')
  output.write(resp.content)
  output.close()

sheet2csv(
    'https://docs.google.com/spreadsheets/d/1kCwlP_LmXQm7DBagptRPHVBBQDUMmbAAhap0nKyVFyE/export?format=csv&id=1kCwlP_LmXQm7DBagptRPHVBBQDUMmbAAhap0nKyVFyE&gid=209928702',
    'combine.csv'
)

In [ ]:
pip install transformers

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModel, get_scheduler, AdamW

In [ ]:
import numpy as np
import pandas as pd
import re
from spacy.lang.id import STOP_WORDS
from tqdm import tqdm

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch import tensor, nn, optim
import torch


# Preprocessing Data

In [ ]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', text)

def clean(text):
  text = str(text)
  text = text.replace('&amp;', ' ')
  text = text.lower()
  text = deEmojify(text)
  text = re.sub(r'@[0-9a-z]+', ' [USERNAME] ', text)
  text = re.sub(r'#[0-9a-z]+', ' [HASHTAG] ', text)
  text = re.sub(r'http\S+', ' [URL] ', text)
  text = re.sub(r'[-]', '', text)
  text = re.sub(r'[^0-9a-z]', ' ', text)
  text = re.sub(r'[0-9]+', ' ', text)
  return text

def clean_stopwords(text, stop_words=STOP_WORDS):
  text = str(text)
  text = text.split()
  text = [t for t in text if t not in stop_words]
  return ' '.join(text)

In [ ]:
df = pd.read_csv('/content/combine.csv')[['TWEET', 'Arah Voting']]
df = df[~df['Arah Voting'].isnull()]
df['Arah Voting'] = df['Arah Voting'].map({'S': 3, 'P':0, 'O':1, 'N':2})
df['TWEET'] = df['TWEET'].apply(clean).apply(clean_stopwords)
df.head()

,TWEET,Arah Voting
0,yixing menerima sumbanganmu pencegahan penyeba...,3
1,suga bts donasikan rp miliar korban virus koro...,3
2,d tanggal february donasi terkumpul rp berhubu...,3
3,kayaknya kampanye yg ngebantu korban virus kor...,3
4,cuman ngasih tau org yg namanya bryan emg ky v...,3


In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(df['TWEET'], df['Arah Voting'], test_size=.2)


In [ ]:
max(df['TWEET'].str.split().apply(len))

49

In [ ]:
class CovidDataset(Dataset):
  def __init__(self, text, label):
    self.text = text
    self.label = label
    self.tokenizer = AutoTokenizer.from_pretrained('indolem/indobert-base-uncased')
    self.encode = self.tokenizer(self.text, max_length=101, 
                     padding='max_length', return_tensors='pt')

  def __getitem__(self, idx):
    return {k:v[idx] for k, v in self.encode.items()} , tensor(self.label[idx])

  def __len__(self):
    return len(self.label)

train_dataset = CovidDataset(list(train_texts.values), train_labels.values)
val_dataset = CovidDataset(list(val_texts.values), val_labels.values)

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/234k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

# Load Model

In [ ]:
class CovidModel(nn.Module):
  def __init__(self):
    super(CovidModel, self).__init__()
    self.indobert = AutoModel.from_pretrained("indolem/indobert-base-uncased")
    self.conv = nn.Sequential(
        nn.Conv1d(101, 256, (7,)),
        nn.Conv1d(256, 128, (7,)),
        nn.AvgPool1d(116)
    )
    self.seq = nn.Sequential(
        nn.Linear(128, 32),
        nn.ReLU(),
        nn.Linear(32, 4),
        nn.ReLU()
    )
    self.lstm = nn.LSTM(768, 128, 2, batch_first=True)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x = self.indobert(**x)
    x, _ = self.lstm(x['last_hidden_state'])
    x = self.conv(x)
    x = x.squeeze()
    x = self.seq(x)
    return self.sigmoid(x)

In [ ]:
criterion = nn.CrossEntropyLoss()
torch.cuda.empty_cache()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = CovidModel()
model.to(device)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
n_epochs = 73
num_training_steps = n_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optim,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

AttributeError: ignored

In [ ]:
for epoch in range(n_epochs):
  losses = []
  total = 0.0
  model.train()
  for batch in tqdm(train_loader, desc='{}/{} epochs'.format(epoch+1, n_epochs)):
    X, y = batch
    X = {k: v.to(device) for k, v in X.items()}
    y = y.to(device)

    outputs = model(X)
    loss = criterion(outputs, y)
    losses.append(loss)
    loss.backward()

    total = total + sum(outputs.argmax(1) == y)

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    
  print('loss :', round(torch.mean(tensor(losses)).item(), 7), end=' ')
  print('| acc :', round(total.item()/len(train_dataset), 4), end=' ')

  model.eval()
  total = 0.0
  for batch in val_loader:
    X, y = batch
    X = {k: v.to(device) for k, v in X.items()}
    y = y.to(device)
    outputs = model(X)
    total = total + sum(outputs.argmax(1) == y)

  print('| val_acc :', round(total.item()/len(val_dataset), 4))
  

1/73 epochs: 100%|██████████| 94/94 [00:54<00:00,  1.71it/s]


loss : 1.3716196 | acc : 0.428 | val_acc : 0.4813


2/73 epochs: 100%|██████████| 94/94 [00:54<00:00,  1.72it/s]


loss : 1.3553642 | acc : 0.428 | val_acc : 0.4813


3/73 epochs: 100%|██████████| 94/94 [00:54<00:00,  1.72it/s]


...

loss : 1.3219684 | acc : 0.428 | val_acc : 0.4813


72/73 epochs: 100%|██████████| 94/94 [00:54<00:00,  1.72it/s]


loss : 1.323431 | acc : 0.428 | val_acc : 0.4813


73/73 epochs: 100%|██████████| 94/94 [00:54<00:00,  1.73it/s]


loss : 1.3226994 | acc : 0.428 | val_acc : 0.4813
